In [1]:
import boto3
import pandas as pd
import numpy as np
from boto3 import client
import os
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [2]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

airflow-data-maymobility.com
amazon-connect-b275969a4e94
annotations.simdata.maymobility.com
applied.maymobility.com
artifacts.code.maymobility.com
assetprotection.maymobility.com
audit-log.maymobility.com
auto-snipping
aws-simplewebstreaming-hostingbucket-783236726606-dev
backup.maymobility.com
cdn.code.maymobility.com
cf-templates-18g0p5n6agm8r-us-east-1
cloudtrail-logs-maymobility
compressed-logs-may
config-bucket-783236726606
crl.its.maymobility.com
dashboard.maymobility.com
data-pipeline.maymobility.com
dependency-proxy.code.maymobility.com
docs.api.maymobility.com
eks-quickstart-lambdazips-us-east-2-783236726606
external-diffs.code.maymobility.com
gitlab-runner-cache.maymobility.com
global.maymobility.com
gsuite-archive.maymobility.com
help.may.network
indy-snow
interventions-maymobility
it-aws-backup.maymobility.com
ivs-log.sdi.maymobility.com
ivs-webrtc-codebuild-e5684116378d
japan-data-pipeline-maymobility
japan-logs-maymobility
lfs-objects.code.maymobility.com
live.its.maymob

In [4]:
# parameters
buck_name = 'log.maymobility.com'
conn = client('s3')
bucket = s3.Bucket(buck_name)
folder_name = ['columbus','detroit','providence']
early_date = {'columbus':'2018-11-06','detroit':'2018-10-29','providence':'2019-02-18'}

In [63]:
columbus_key = []
detroit_key = []
providence_key = []
for obj in bucket.objects.all():
    key = obj.key
    if ('columbus' in key):
        columbus_key.append(key)
    elif ('detroit' in key):
        detroit_key.append(key)
    elif ('providence' in key):
        providence_key.append(key)

In [136]:
columbus = pd.DataFrame([x.split('/') for x in columbus_key])
detroit = pd.DataFrame([x.split('/') for x in detroit_key])
providence= pd.DataFrame([x.split('/') for x in providence_key])

In [210]:
not_time_list = ['config',
 'dev-retrea',
 'exhaust-lo',
 'keep_long_',
 'map-ref-lo',
 'mcmlog_201',
 'mcmlog_202',
 'ops_logs',
 'providence',
 'ogs',
 'may',
 'snippets',
 'snips',
 'syslogs','logs']

In [211]:
def select_pre_logs(site_name,data):
    data[0] = data[0].str.strip()
    data['folder_time'] = data[0].str.slice(0,10)
    data = data[~data['folder_time'].isin(not_time_list)] # drop no time one
    data['folder_time'] = data.folder_time.str.replace('_','-')
    data.folder_time = pd.to_datetime(data.folder_time)# change time type
    data['pre'] = np.array(data.folder_time < pd.to_datetime(early_date[site_name]))
    return data

In [212]:
columbus_pre = select_pre_logs('columbus',columbus)
detroit_pre = select_pre_logs('detroit',detroit)
providence_pre = select_pre_logs('providence',providence)

/var/folders/57/p4z_h0711cx5h0by91hbgqdc0000gp/T/ipykernel_24920/3474141406.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['folder_time'] = data.folder_time.str.replace('_','-')
/var/folders/57/p4z_h0711cx5h0by91hbgqdc0000gp/T/ipykernel_24920/3474141406.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pre'] = np.array(data.folder_time < pd.to_datetime(early_date[site_name]))


In [251]:
columbus_pre = columbus_pre[[0,1,2]].fillna('')
columbus_pre = columbus_pre[[0,1,2]].agg('/'.join, axis=1)
detroit_pre = detroit_pre[[0,1,2]].fillna('')
detroit_pre = detroit_pre[[0,1,2]].agg('/'.join, axis=1)
providence_pre = providence_pre[[0,1,2]].fillna('')
providence_pre = providence_pre[[0,1,2]].agg('/'.join, axis=1)

In [256]:
columbus_pre = columbus_pre.drop_duplicates()
detroit_pre = detroit_pre.drop_duplicates()
providence_pre = providence_pre.drop_duplicates()

In [262]:
columbus_pre = pd.DataFrame(columbus_pre,columns = ['log_name'])
detroit_pre = pd.DataFrame(detroit_pre,columns = ['log_name'])
providence_pre = pd.DataFrame(providence_pre,columns = ['log_name'])

In [263]:
columbus_pre.to_csv('columbus_pre.csv')
detroit_pre.to_csv('detroit_pre.csv')
providence_pre.to_csv('providence_pre.csv')